# Time series forecasting methods

In this notebook we'll see 5 models for time series forecasting: 

- Autoregressive Integrated Moving Average (ARIMA)
- Facebook Prophet
- Gradient Boosted Trees with Quantile Regression as the loss function
- Recurrent Neural Network
- Kalman Filter

In particular we'll apply these methods to predict the number of confirmed cases of COVID-19 in Spain. 

Disclaimer: this notebook is not intended to make a serious forecast of the number of cases of this virus, just to see some time series techniques applied to a real world dataset.

Disclaimer: the purpose of this notebook is to just see these models, we won't deal much with the data: understanding it, creating better features, appropriate transformations, etc. We must pay much more attention to the data when doing a real project with some dataset.

In [ ]:
!pip install --quiet pmdarima

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from fbprophet import Prophet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F

np.random.seed(8)
torch.manual_seed(8);

# Comment this during development
import warnings; warnings.filterwarnings(action='ignore')

## Data

The data comes from the Kaggle dataset ["COVID-19 Dataset"](https://www.kaggle.com/imdevskp/corona-virus-report), which, in turns, comes from the [Github repository of John Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series).

The original data contains the number of confirmed cases and fatalities of each country and it is updated constantly. However, for simplicity we'll consider just the number of confirmed cases in Spain from February 25th to May 10th. Also, there were a few negative values for some reason. For simplicity, those were replaced by zeros.

In [ ]:
DATA_PATH = "/kaggle/input/corona-virus-report/"

df_spain = pd.read_csv(DATA_PATH + "covid_19_clean_complete.csv", parse_dates=["Date"])

df_spain = df_spain[(df_spain["Country/Region"] == "Spain") & 
                    (df_spain.Date >= "2020-02-24") &
                    (df_spain.Date <= "2020-05-10")]    # because the dataset is constantly being updated
df_spain = (df_spain
            .loc[:, ["Date", "Confirmed"]]  # only interested in confirmed cases
            .rename(columns={"Confirmed": "TargetValue"})
            .set_index("Date")
            .diff()         # convert to daily cases
            .iloc[1:]       # remove first row, it is NaN due to the .diff()
            .clip(lower=0)) # remove <0 values

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_spain.index, y=df_spain.TargetValue, mode='lines+markers'))
fig.update_layout(hovermode="x",
                  title="Confirmed daily cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of confirmed daily cases")

fig.show()

## Models

### Autoregressive Integrated Moving Average (ARIMA)

An ARIMA model is used to fit to a time series that is stationary (time series data whose statistical properties like the mean and variance don't change over time) or that can be made stationary by differencing (create a new time series by taking the difference of a value with the previous one, assuming that the time steps are of equal length we can say that this operation is equivalent to a discrete derivative).


An ARIMA model consists of three parts:

- (I) The differencing step: this just applies the differencing operation to the time series one or more times. The number of times this operation is applied is determined by a parameter $d$

- (AR) An Autoregressive model: this is a model that approximates the value for time $t$ as a linear combination of values from the $p$ previous time steps, where $p$ is a parameter that has to be specified by the user. The rest of the parameters of the model are estimated using methods like maximum likelihood estimation.

- (MA) A Moving Average model: this is a model that approximates the value for time $t$ as the mean of the series plus a linear combination of errors from the $q$ previous steps. Just like the AR model, the number of previous time steps to take into account is a parameter that has to be specified by the user and the rest are estimated.
    
Thus, ARIMA is the sum of an AR model and a MA model and they are fitted to a time series differentiated $d$ times. In total, there are three parameters that have to be specified by the user: $p$, $d$, $q$. There are methods to guide the manual search for good parameters. But we can also do it iteratively, by performing a search over the parameter space and returning the model with the best result. This result is commonly measured by the Akaike Information Criterion (AIC), which is a combination of the likelihood that the model describes the data and a penalty for the number of parameters of the model (penalizing the complexity of the model).

To evaluate the model we first divided the data into a training set with data until April 21th and a testing set with the rest of the data. We used auto ARIMA to look for the best parameters iteratively within the training set. Finally we used the model to forecast many time steps into the future and below we can see the comparison between the forecasts and the real test data.

In [ ]:
def fit_predict_auto_arima(df_train, df_test):
    arima = pm.auto_arima(df_train, start_p=1, start_q=1)
    forecasts, forecasts_pred_interval = arima.predict(df_test.shape[0], 
                                                       return_conf_int=True, 
                                                       alpha=0.05)

    forecasts = pd.Series(data=forecasts, index=df_test.index)
    forecasts_pred_interval = pd.DataFrame(data=forecasts_pred_interval, 
                                           index=df_test.index,
                                           columns=["low", "high"])

    return forecasts, forecasts_pred_interval, arima.order

df_train, df_test = train_test_split(df_spain)
forecasts, forecasts_pred_interval, forecast_arima_order = fit_predict_auto_arima(df_train, df_test)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index, y=df_train.TargetValue, 
                         mode='lines+markers', name="train data"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.TargetValue, 
                         mode='lines+markers', name="test data"))
fig.add_trace(go.Scatter(x=df_test.index, y=forecasts, 
                         mode='lines+markers', name="forecast"))
fig.add_trace(go.Scatter(x=df_test.index, y=forecasts_pred_interval.low, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="lower prediction interval"))
fig.add_trace(go.Scatter(x=df_test.index, y=forecasts_pred_interval.high, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="upper prediction interval"))

fig.update_layout(hovermode="x",
                  title=f"ARIMA{forecast_arima_order} applied to the number of confirmed cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of confirmed daily cases")

fig.show()

Trying to predict the raw number of confirmed cases is not an easy task. Taking into account that the third value of the test set was a negative number, the predictions for the first four days are not bad. After that the model cannot predict much variation with respect to previous days, this shows that ARIMA models tend to be better for making short-term predictions.

### Facebook Prophet

Prophet is a forecasting tool open sourced by Facebook meant to be easy to use. This model first tries to fit a curve to the data to find the overall trend of the series. Then, it looks for seasonalities and adds them to the forecast. Finally, it adds the effects of holidays.

> At its core, the Prophet procedure is an additive regression model with four main components:
>- A piecewise linear or logistic growth curve trend. Prophet automatically detects changes in trends by selecting changepoints from the data.
>- A yearly seasonal component modeled using Fourier series.
>- A weekly seasonal component using dummy variables.
>- A user-provided list of important holidays.

(On newer versions, daily and custom seasonalities can be applied included too)

> The important idea in Prophet is that by doing a better job of fitting the trend component very flexibly, we more accurately model seasonality and the result is a more accurate forecast. We prefer to use a very flexible regression model (somewhat like curve-fitting) instead of a traditional time series model for this task because it gives us more modeling flexibility, makes it easier to fit the model, and handles missing data or outliers more gracefully.

(The cites are from Facebook Research: https://research.fb.com/blog/2017/02/prophet-forecasting-at-scale/)

Just like with the ARIMA model, here we fit the model to the training data and use it to predict many steps into the future. The model was fitted with just an added a seasonality for every two weeks. I thought that this would be a good idea since because that is the maximum estimated incubation period of the virus.

In [ ]:
def fit_predict_prophet(df_train, df_test):

    df_temp = df_train.reset_index().rename(columns={
      'Date': 'ds',
      'TargetValue': 'y'
    })

    m = (Prophet(daily_seasonality=False,    # Don't add a daily seasonal component
                 yearly_seasonality=False,   # Don't add a yearly seasonal component
                 weekly_seasonality=False,   # Don't add a weekly seasonal component
                 changepoint_prior_scale=2)  # Make the model more flexible to trend changes
      .add_seasonality(name="twoweek", 
                       period=14,          # Model a seasonal component every two week
                       fourier_order=2)    # More orders -> more periods in seasonality -> overfit
      .add_country_holidays("Spain"))      # Add one-hot encoded variables for holidays

    m.fit(df_temp)

    future = m.make_future_dataframe(periods=df_test.shape[0])
    forecast = m.predict(future)
    forecast["ds"] = pd.to_datetime(forecast.ds)

    return forecast.set_index("ds").iloc[-df_test.shape[0]:]

forecasts_prophet = fit_predict_prophet(df_train, df_test)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index, y=df_train.TargetValue, 
                         mode='lines+markers', name="train data"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.TargetValue, 
                         mode='lines+markers', name="test data"))
fig.add_trace(go.Scatter(x=df_test.index, y=forecasts_prophet.yhat, 
                         mode='lines+markers', name="forecast"))
fig.add_trace(go.Scatter(x=df_test.index, y=forecasts_prophet.yhat_lower, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="lower prediction interval"))
fig.add_trace(go.Scatter(x=df_test.index, y=forecasts_prophet.yhat_upper, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="upper prediction interval"))

fig.update_layout(hovermode="x",
                  title="Prophet applied to the number of confirmed cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of confirmed daily cases")

fig.show()

Facebook Prophet was mainly designed for business problems. Though it is very simple, so we don't lose much time trying to use it on other types of time series. And here seems to have captured the descending trend. Though this was not the case at first, this is thanks to that we can make the model more flexible to capture trend changes.

### Gradient Boosted Trees with Quantile Regression as the loss function

Gradient Boosting is a technique that produces a model based on weaker models, usually decision trees. 

At a very high level, given a loss function, the training part of this technique starts with a constant value as the initial prediction. This value is the one that minimizes the loss function, it usually is the mean in case of regression because the most used loss function is the Mean Squared Error. Then, we calculate the _pseudo-residuals_ of the target values and the prediction. After that we fit a tree with having the residuals as target values and we assign as leaf values the constant that minimizes the loss function. And the process repeats, having the results of each tree weighted by a factor called the learning rate.

Note that the predictions of a Gradient Boosting model are constants values learned from the data, usually the mean. Thus, these models cannot extrapolate to data outside the range of the training data. Because of this, these models have to be used with extra care when dealing with time series data that might have future values out of the range.

In this case, we decided to use Gradient Boosting because we don't expect the future values to go out of the range of the training data in the near future. And because this is one model that has already implemented the Quantile Loss as loss function in scikit-learn. Other models that minimizes a loss function can be used instead, for instance Linear Regression and Neural Networks.

The mean of the data is the constant that minimizes the Mean Squared Error. The constant that minimizes the Quantile Loss is a percentile of the data. When used as a loss function, we can use the model to predict a high percentile and a lower one to get a prediction interval.

To train this model for a time series, or any other Machine Learning model, we have to create features so the model can try to capture the time component. In particular, datetime features were created, for instance, day of the month, month, day of the week and so on. Another type of feature used were "lag" features: the number of cases in the previous 14 days.

Three models were trained for the 2.5, 50 and 97.5 percentiles respectively. This way we have a prediction for the  median and a 95% prediction interval.

The previous models were used to predict many time steps ahead without any information of the test data at inference time. This changes for this model and the rest: we'll keep training just on the training data, but for simplicity, to predict the value for a time step $t$, these models will have as input the information of the previous days. So these models are used to predict just one time step ahead.

In [ ]:
X_train = df_spain.reset_index()

# Add datetime features
X_train["Month"] = X_train.Date.dt.month
X_train["Week"] = X_train.Date.dt.week
X_train["Day"] = X_train.Date.dt.day
X_train["Dayofweek"] = X_train.Date.dt.dayofweek
X_train["IsWeekEnd"] = X_train.Date.dt.dayofweek.isin([5, 6]).astype(int)
X_train.drop("Date", axis=1, inplace=True)

# Add lag features
N_SHIFTS = 14
X_train["TargetValue"] = X_train.TargetValue
for i in range(N_SHIFTS):
    X_train[f"TargetShift{i+1}"] = X_train.TargetValue.shift(i+1).fillna(0)

X_train = pd.get_dummies(X_train, columns=['Month', 'Week', 'Day', 'Dayofweek', 'IsWeekEnd'])

y_train = X_train.TargetValue
X_train = X_train.drop("TargetValue", axis=1)

n_test = df_test.shape[0]
X_train = X_train.iloc[:-n_test]
y_train = y_train.iloc[:-n_test]
X_valid = X_train.iloc[-n_test:]
y_valid = y_train.iloc[-n_test:]

In [ ]:
model_percentile_975 = GradientBoostingRegressor(loss='quantile', alpha=0.975, random_state=8)
model_percentile_50  = GradientBoostingRegressor(loss='quantile', alpha=0.5, random_state=8)
model_percentile_025  = GradientBoostingRegressor(loss='quantile', alpha=0.025, random_state=8)

model_percentile_50.fit(X_train, y_train)
model_percentile_025.fit(X_train, y_train)
model_percentile_975.fit(X_train, y_train);

In [ ]:
pred_975 = model_percentile_975.predict(X_valid)
pred_50  = model_percentile_50.predict(X_valid)
pred_025 = model_percentile_025.predict(X_valid)

pred_50 = pd.Series(pred_50, index=df_test.index)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index, y=df_train.TargetValue, 
                         mode='lines+markers', name="train data"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.TargetValue, 
                         mode='lines+markers', name="test data"))
fig.add_trace(go.Scatter(x=df_test.index, y=pred_50, 
                         mode='lines+markers', name="forecast"))
fig.add_trace(go.Scatter(x=df_test.index, y=pred_025, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="lower prediction interval"))
fig.add_trace(go.Scatter(x=df_test.index, y=pred_975, 
                         mode="lines", fill='tonexty', line_color="lightgreen", name="upper prediction interval"))

fig.update_layout(hovermode="x",
                  title="Gradient Boosting applied to the number of confirmed cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of confirmed daily cases")

fig.show()

The model doesn't seem to be sure about the predictions. However, it is interesting to see that the prediction interval, although quite wide, includes most of the real number of cases. Also, it doesn't go below zero because there were no negative values in the training set.

### Recurrent Neural Network (RNN)

Recurrent Neural Networks take a sequence as input. The first value of the sequence is multiplied by some learnable factors and passed to a non-linear function. Then, the result is given as output. For the next item in the sequence, the input is multiplied by the same learnable factors and the the result of the previous step is also added. Then, this is passed to the same non-linear function and the process continues in the same way for the rest of the sequence. Finally, all the outputs can be passed to a "linear layer" to get a single output. That is, take a linear combination of the outputs of the RNN "layer" with some learnable parameters and pass it to a final non-linear function.

To learn those parameters, first, a loss function is defined. Then, the derivative of the loss can be calculated with respect to each parameter. Finally, the parameters can be updated via a variant of Gradient Descent.

To train the network, the same train/test split as before was used. Though, this time the data had to be converted to sequences. For that, a sequence of length three was enough (with the small amount of data, a larger sequence length would result in having less data). Also, the data was scaled to be in [0, 1] range so Gradient Descent converges faster.

In [ ]:
def create_sequences(data, length):
    xs = [data[i:(i + length)] for i in range(data.shape[0] - length - 1)]
    ys = [data[(i + length)] for i in range(data.shape[0] - length - 1)]

    xs = np.array(xs)
    ys = np.array(ys)

    xs = torch.from_numpy(xs).float().view(-1, length, 1)
    ys = torch.from_numpy(ys).float().view(-1, 1)

    return xs, ys

In [ ]:
train = df_train.TargetValue.values.reshape(-1, 1)
test = df_test.TargetValue.values.reshape(-1, 1)

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

sequence_length = 3

X_train, y_train = create_sequences(train, sequence_length)
X_test, y_test = create_sequences(test, sequence_length)

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size=1, output_size=1,
               hidden_dim=10, num_layers=1, 
               sequence_length=sequence_length):
        """
          Good explanation:
            https://towardsdatascience.com/pytorch-basics-how-to-train-your-neural-net-intro-to-rnn-cb6ebc594677

          input_size: 
            number of features of the input: 1 in our case because we are dealing 
            with number of cases. Could be more, for example, for one-hot encoded 
            input

          output_size:
            1 because we want to predict just the number of cases that follow
            the sequence

          num_layers:
            number of stacked RNN layers (if more than one, it means that the 
            output of the first RNN layer will pass to another RNN and so on)

          sequence_length:
            number of previous values to take into account

        """
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim * sequence_length, output_size)

    def forward(self, sequences):
        # (batch_size, sequence_length, input_size)
        sequences = sequences.view(-1, self.sequence_length, self.input_size)

        # Dummy hidden state for first input
        hidden = self.initial_hidden(sequences.shape[0])

        # Pass through the RNN layer
        out, hidden = self.rnn(sequences, hidden)

        # Flatten the output and pass it to the linear layer
        out = out.flatten(start_dim=1)
        out = self.fc(out)

        return out, hidden

    def initial_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim)

In [ ]:
model_rnn = RNN()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(lr=5e-3, params=model_rnn.parameters())

def train(num_epochs, model, optimizer, loss_fn, verbose=False):
    for epoch in range(1, num_epochs + 1):
        optimizer.zero_grad()
        output, _ = model_rnn(X_train)
        loss = loss_fn(output, y_train)

        with torch.no_grad():
            y_test_pred, _ = model(X_test)
            test_loss = loss_fn(y_test_pred.float(), y_test)

        loss.backward()
        optimizer.step()

        if verbose and epoch%10 == 0:
            print(f'Epoch: {epoch}/{num_epochs}. Train Loss: {loss.item():.3f}. Test loss: {test_loss.item(): .3f}')

train(100, model_rnn, optimizer, loss_fn)

In [ ]:
with torch.no_grad():
    model_rnn.eval()
    out, _ = model_rnn(X_test)
    out = scaler.inverse_transform(out.numpy().reshape(1, -1)).reshape(-1)

    tr_out, _ = model_rnn(X_train)
    tr_out = scaler.inverse_transform(tr_out.numpy().reshape(1, -1)).reshape(-1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_train.index, y=df_train.TargetValue, 
                         mode='lines+markers', name="train data"))
fig.add_trace(go.Scatter(x=df_train.index[-tr_out.shape[0]:], y=tr_out, 
                         mode='lines+markers', name="train forecast"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.TargetValue, 
                         mode='lines+markers', name="test data"))
fig.add_trace(go.Scatter(x=df_test.index[-out.shape[0]:], y=out, 
                         mode='lines+markers', name="test forecast"))

fig.update_layout(hovermode="x",
                  title="RNN applied to the number of confirmed cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of confirmed daily cases")

fig.show()

The predictions for the test set look very close to the real data. So the predictions for the training set were also plotted and looks like maybe the network learned to return some kind of average of the previous values.

### Kalman Filter

We can have a model of how a dynamic system changes over time. Lets say that we want to measure the position and velocity of a vehicle. With Newton's laws we can estimate where the vehicle will be at a certain timestep given an initial state. However, real world is not simple, so our estimate will deviate from the true state of the vehicle due to other factors. The road could be slippery, there could be traffic and so on. 

Thankfully we have sensors and we can add the information given from them to correct our estimates over time. However, these sensors also come with some uncertainty. That is why we don't rely only on them.

The Kalman filter is a method that combines uncertain information to get a better estimate of the unknown state of a system. 

Just like the Newton's laws with the vehicle example, we have simple models that represent the spread of a virus. And we can also consider the number of daily cases as measurements of the spread of the virus. So we can try to use a Kalman filter to estimate the current state of the spread and get a prediction for the next state.

In a Kalman filter we start with an initial state. Then we use our knowledge of the system dynamics to get a prediction for the next state. Then, we get a measure and use this information to update/correct our belief of the current state. After that, we make another prediction and the process repeats.

To do this, a Kalman filter uses 5 equations:

#### Prediction equations
To make a prediction we use two equations. Because a Kalman filter has to deal with uncertainty, it assumes that the variables of the state are random with a normal distribution. Thus, we have to keep track of two pieces of information: the means and covariances.


##### State Extrapolation
$$ \hat{x}^-_t = \mathbf{F}\hat{x}^-_{t-1} + \mathbf{G}\hat{u}^-_{t-1} $$

$ \hat{x}^-_t $ is the __mean__ estimate of the state at time $t$ before taking into account the measurement information (that is represented by the $^-$ superscript). 

This equation tries to be a general framework for us to describe the system dynamics that governs the system. It has two parts:

- $ \mathbf{F}\hat{x}^-_{t-1} $ is to describe how the system changes. For example, if our state is the position and velocity of a vehicle, we can define the $\mathbf{F}$ matrix to update the position based on the velocity following Newton's laws and keep the velocity constant.

- $ \mathbf{G}\hat{u}^-_{t-1} $ is to describe external factors that we know. For example, in a vehicle, the driver can apply an acceleration to the vehicle with the gas pedal or brake.


##### Covariance Extrapolation

$$ \mathbf{P}^-_t = \mathbf{F}\mathbf{P}^-_{t-1}\mathbf{F}^\top + \mathbf{Q}$$

Like we said earlier, a Kalman filter deals with random normal variables. The State Extrapolation equation deals with predicting the new mean of the state, this one with predicting the new covariances. It also has two parts:

- To understand the first part we need to remember that the variance of a variable scaled by a factor is the same as the variance times the square of the factor: $ V(aX) = a^2V(X) $. And the same applies for variables in matrix format. So, before our covariance matrix was $ \mathbf{P}^-_{t-1} $, after applying the transformation $ \mathbf{F} $ to the random variable, we have $ \mathbf{F}\mathbf{P}^-_{t-1}\mathbf{F}^\top $

- $\mathbf{Q}$ is just random noise to account for any external known or unknown uncertainty.


#### Update/correction equations

At this point a time step has passed by. From the previous predictions we have an estimate of the current state and we receive a measurement. Now we want to combine this information to get a more accurate estimate of the current state. Because sensors are noisy, we also use random normal variables to represent the measurements.

Now we have two random variables that represent two different guesses for the system state. One comes from our prediction and the other one from the measures. To get the most likely state we can just multiply the two variables (when we want to know the probability of two events occurring we multiply the probability of both events).

Knowing the probability density function of a normal distribution we can calculate the product and get a formula for the mean and covariance matrix of the new variable. This is where the three update formulas come from.

![Image of two normal distributions and the result of the product](https://www.bzarg.com/wp-content/uploads/2015/08/gauss_joint.png)

(Image from the article ["How a Kalman filter works, in picture" posted on Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/))


##### State Update/Correction

$$ \hat{x}_t = \hat{x}^-_t + \mathbf{K}_t ( z_t - \mathbf{H}\hat{x}^-_t ) $$

- $ z_t $ are the measurements taken at time $ t $. 
- $\mathbf{H}$ is the transformation to apply to the state to have it in the same units as the measurements.
- $ \mathbf{K} $ is the Kalman Gain defined below

##### Covariance Update/Correction

$$ \mathbf{P}_t = \mathbf{P}^-_t - \mathbf{K}_t \mathbf{H} \mathbf{P}^-_t $$


##### Kalman Gain

$$ \mathbf{K}_t = \mathbf{P}^-_t \mathbf{H}^\top ( \mathbf{H} \mathbf{P}^-_t \mathbf{H}^\top + \mathbf{R}_t)^{-1} $$

- $ \mathbf{R}_t $ is the covariance matrix associated to the measurements


In [ ]:
class KalmanFilter:
    def __init__(self, x, F, P, Q, H, u=None, G=None):
        self.x = x
        self.F = F
        self.P = P
        self.Q = Q
        self.H = H
        self.u = u
        self.G = G

        if G is None or u is None:
            self.u = np.zeros_like(x)
            self.G = np.zeros_like(F)

        self.predict()


    def predict(self):
        self.x = self.F @ self.x + self.G @ self.u
        self.P = self.F @ self.P @ self.F.T + self.Q


    def update(self, z, R):
        K = self.P + self.H.T @ np.linalg.inv(self.H @ self.P @ self.H.T + R)
        self.x = self.x + K @ (z - self.H @ self.x)
        self.P = self.P - K @ self.H @ self.P


class KalmanFilter1D:
    def __init__(self, x, F, P, Q):
        self.x = x
        self.F = F
        self.P = P
        self.Q = Q

        self.predict()


    def predict(self):
        self.x = self.F * self.x
        self.P = self.P * self.F**2  + self.Q


    def update(self, z, R):
        K = self.P / (self.P + R)
        self.x = self.x + K * (z - self.x)
        self.P = self.P - K * self.P

Because the Kalman Filter is not a model that has to be trained, I decided to apply it to the whole data and plot the predictions for the next states. The dynamics were chosen to be very simple: the number of total confirmed cases for time $t$ will be the same as for time $t-1$. Also, the variances were chosen a bit arbitrarily.

In [ ]:
x = 0
F = 1
P = 1
Q = 50
zs = df_spain.cumsum().values.reshape(-1)
Rs = np.ones_like(zs) * 100

kalman = KalmanFilter1D(x, F, P, Q)
corrections = []
predictions = []
verbose = False

for t, (z, R) in enumerate(zip(zs, Rs)):
    predictions.append(kalman.x)
    if verbose: print(f"Time {t+1}: Prediction for current state: {kalman.x:.4f}.")

    kalman.update(z, R)
    corrections.append(kalman.x)
    if verbose: print(f"Time {t+1}: Measurement: {z:.4f}. Correction: {kalman.x:.4f}")

    kalman.predict()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_spain.index, y=zs,
                    mode='lines+markers',
                    name='measurements'))
fig.add_trace(go.Scatter(x=df_spain.index, y=predictions,
                    mode='lines+markers',
                    name='predictions'))
#fig.add_trace(go.Scatter(x=df_spain.index, y=corrections,
#                    mode='lines+markers', 
#                    name='after corrections'))

fig.update_layout(hovermode="x",
                  title="Kalman Filter applied to the total number of confirmed cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of total confirmed cases")

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_spain.index[1:], y=np.diff(zs).astype(int),
                    mode='lines+markers',
                    name='measurements'))
fig.add_trace(go.Scatter(x=df_spain.index[1:], y=np.diff(predictions).astype(int),
                    mode='lines+markers',
                    name='predictions'))
#fig.add_trace(go.Scatter(x=df_spain.index[1:], y=np.diff(corrections).astype(int),
#                    mode='lines+markers', 
#                    name='after corrections'))
fig.update_layout(hovermode="x",
                  title="Kalman Filter applied to the number of daily confirmed cases in Spain",
                  xaxis_title="Date",
                  yaxis_title="Number of daily confirmed cases")

fig.show()

Even though the assumption of the dynamics is very naive, the Kalman Filter doesn't do a terrible job approximating the number of cases. Maybe using another version of the Kalman Filter we could use some of the previous models to predict the next states and have better results.

## References

- "Practical Time Series Analysis". Author: Aileen Nielsen. ISBN: 9781492041658

- "Prophet: forecasting at scale". From: Facebook Research. URL: https://research.fb.com/blog/2017/02/prophet-forecasting-at-scale/. Last day visited: 25 May 2020.

- "5 Regression Loss Functions All Machine Learners Should Know". Author: Prince Grover. URL: https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0. Last day visited: 25 May 2020.

- "How a Kalman filter works, in pictures". From: Bzarg. URL: https://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/. Last day visited: 25 May 2020.

- "Kalman Filter tutorial". Author: Alex Becker. URL:  https://www.kalmanfilter.net/default.aspx. Last day visited: 25 May 2020. 